# Example 11: Variable timestep method
This is an example of **``LFPy``** running in an **``IPython notebook``**. To run through this example code and produce output, press **``<shift-Enter>``** in each code block below.

First step is to **import ``LFPy``** and other packages for analysis and plotting:

In [ ]:
%matplotlib inline

In [ ]:
import LFPy
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [ ]:
# fix seed
np.random.seed(1234)

In [ ]:
# Cell parameters
cell_params = dict(
    morphology='morphologies/L5_Mainen96_LFPy.hoc',
    dt=0.05, # simulation time resolution, will be ignored for variable_dt methods (ms)
    tstop=1000, # simulation time (ms)
    passive=True, # use passive mechanism
)
# Synapse parameters
syn_params = dict(
    idx=0, # compartment index
    syntype='ExpSyn', # exponential conductance synapse type
    record_current=True, # record synaptic current
    e=0, # reversal potential (mV)
    tau=5, # synaptic time constant (ms)
    weight=0.01 # synaptic max conductance (uS)
)

In [ ]:
# synapse activation times (ms)
sptimes = np.random.rand(100)*1000
sptimes = np.sort(sptimes)

In [ ]:
def simulation(variable_dt=False):
    '''
    Parameters
    ----------
    variable_dt: bool
        whether or not variable timestep method is used
    
    Returns
    -------
    tvec: ndarray
        simulation time vector (ms)
    somav: ndarray
        cell somatic voltage (mV)
    syni: ndarray
        synaptic current (nA)
    tictoc: float
        simulation time (s)
    '''
    cell = LFPy.Cell(**cell_params)
    syn = LFPy.Synapse(cell, **syn_params)
    syn.set_spike_times(sptimes=sptimes)
    toc = time()
    cell.simulate(variable_dt=variable_dt)
    tic = time()
    return cell.tvec, cell.somav, syn.i, tic-toc 

In [ ]:
# run simulation, plot comparison of output
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(12, 8))
for variable_dt in [True, False]:
    tvec, somav, syni, tictoc = simulation(variable_dt)
    print('sim time variable_dt={}: {} (ms)'.format(variable_dt, tictoc*1000))
    axes[0].plot(tvec, syni)
    axes[0].set_ylabel(r'$I_\mathrm{syn}$ (nA)')
    axes[1].plot(tvec, somav)
    axes[1].set_ylabel(r'$V_\mathrm{soma}$ (mV)')
    axes[2].step(tvec[:-1], np.diff(tvec), where='pre')
    axes[2].set_ylabel('$\Delta t$ (ms)')
    axes[2].set_xlabel(r'$t$ (ms)')